In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm

/home/chaudhary.ans/.conda/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-11 06:33:37.118217: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-11 06:33:37.131507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733916817.153360   12328 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733916817.160028   12328 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 06:33:

In [2]:
df = pd.read_csv("data/reminiscences_of_a_stock_operator_qa.csv", sep="\t")

In [3]:
df.head()

,question,answer
0,What was your first job in finance?,My first job was as a quotation-board boy at a...
1,What key lessons did you learn from your first...,I learned the importance of quick mental calcu...
2,How did your early skills in math affect your ...,"My strong math skills, especially mental arith..."
3,Describe your first experience making money in...,My first profitable trade was on Burlington. ...
4,What was your initial trading strategy?,My initial strategy focused on recognizing pat...


In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [5]:
class QADataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Set the pad_token to eos_token if not already set
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        question = row['question']
        answer = row['answer']
        
        # Prepare the input for GPT-2
        input_text = f"Question: {question}\nAnswer:"
        target_text = answer
        
        # Tokenize the input and the target (GPT-2 is a causal language model)
        encoding = self.tokenizer(
            input_text,
            target_text,
            truncation=True,
            padding="max_length",  # Enable padding here
            max_length=self.max_length,
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding['input_ids'].squeeze(0),
            "attention_mask": encoding['attention_mask'].squeeze(0),
            "labels": encoding['input_ids'].squeeze(0)  # GPT-2 uses causal language modeling
        }


In [6]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [7]:
MAX_LEN = 128
BATCH_SIZE = 4
EPOCHS = 50
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
train_dataset = QADataset(train_df, tokenizer, MAX_LEN)
test_dataset = QADataset(test_df, tokenizer, MAX_LEN)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [10]:
model.to(DEVICE)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [11]:
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

/home/chaudhary.ans/.conda/envs/myenv/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
def train(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    return total_loss / len(dataloader)

In [13]:
def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(dataloader)

In [14]:
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    train_loss = train(model, train_loader, optimizer, DEVICE)
    val_loss = evaluate(model, test_loader, DEVICE)
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

Epoch 1/50


Evaluating: 100%|██████████| 30/30 [00:04<00:00,  6.23it/s]


Train Loss: 1.4724, Validation Loss: 1.0485
Epoch 2/50


Evaluating: 100%|██████████| 30/30 [00:04<00:00,  6.10it/s]


Train Loss: 1.0392, Validation Loss: 0.9554
Epoch 3/50


...

Train Loss: 0.1020, Validation Loss: 1.2127
Epoch 49/50


Evaluating: 100%|██████████| 30/30 [00:04<00:00,  6.14it/s]


Train Loss: 0.0989, Validation Loss: 1.2086
Epoch 50/50


Evaluating: 100%|██████████| 30/30 [00:05<00:00,  5.77it/s]

Train Loss: 0.0973, Validation Loss: 1.2172


In [21]:
# Save the fine-tuned model
model.save_pretrained("./gpt2-chatbot")
tokenizer.save_pretrained("./gpt2-chatbot")

print("Model fine-tuned and saved!")

Model fine-tuned and saved!


In [22]:
def chat_with_gpt2(model, tokenizer, question, device="cpu"):
    model.eval()
    
    # Prepare the input prompt for the model
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate the answer
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"], 
            max_length=128,  # You can adjust this to control the length of the answer
            num_beams=5,     # Beam search for better quality
            early_stopping=True
        )

    # Decode and return the answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [23]:
# Example usage:
question = "What is the main theme of the book?"
answer = chat_with_gpt2(model, tokenizer, question, device=DEVICE)
print(f"Answer: {answer}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer: Question: What is the main theme of the book?
Answer:The primary theme is greed and fear, the need to avoid overtrading and impulsive decision-making, and the need to maintain discipline and composure in the face of market manipulation.  It's about understanding the combination of greed and fear with rational analysis and decisive action.


In [24]:
# Example usage:
question = "What is the correct time to enter the market?"
answer = chat_with_gpt2(model, tokenizer, question, device=DEVICE)
print(f"Answer: {answer}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer: Question: What is the correct time to enter the market?
Answer:The correct time to enter the market is when you have a good understanding of market trends and a good understanding of the fundamental reasons behind price movements.  You must also be prepared to make rational, well-timed trades to capitalize on the market's movements.


In [25]:
# Example usage:
question = "What was your first job in finance?"
answer = chat_with_gpt2(model, tokenizer, question, device=DEVICE)
print(f"Answer: {answer}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer: Question: What was your first job in finance?
Answer:My first job was as a quotation-board boy at a stock brokerage firm. I was quick with numbers and excelled at mental arithmetic, skills that proved invaluable later in my career.
